# refactoring the bsuccessors

In [28]:
# -----------------
# User Instructions
# 
# In this problem you will be refactoring the bsuccessors function.
# Your new function, bsuccessors3, will take a state as an input
# and return a dict of {state:action} pairs. 
#
# A state is a (here, there, light) tuple. Here and there are 
# frozensets of people (each person is represented by an integer
# which corresponds to their travel time), and light is 0 if 
# it is on the `here` side and 1 if it is on the `there` side.
#
# An action is a tuple of (travelers, arrow), where the arrow is
# '->' or '<-'. See the test() function below for some examples
# of what your function's input and output should look like.

def bsuccessors3(state):
    """Return a dict of {state:action} pairs.  State is (here, there, light)
    where here and there are frozen sets of people, light is 0 if the light is 
    on the here side and 1 if it is on the there side.
    Action is a tuple (travelers, arrow) where arrow is '->' or '<-'"""
    here, there, t = state
    d = {}
    if t == 0:
        return dict(((here - frozenset([a, b]), there | frozenset([a, b]), 1), (set([a, b]), '->'))
                    for a in here 
                    for b in here)
    else:
        for a in there:
            for b in there:
                state = here | frozenset([a, b]), there - frozenset([a, b]), 0
                action = set([a, b]), '<-'
                d[state] = action 
    return d

def test():
    assert bsuccessors3((frozenset([1]), frozenset([]), 0)) == {
            (frozenset([]), frozenset([1]), 1)  :  (set([1]), '->')}

    assert bsuccessors3((frozenset([1, 2]), frozenset([]), 0)) == {
            (frozenset([1]), frozenset([2]), 1)    :  (set([2]), '->'), 
            (frozenset([]), frozenset([1, 2]), 1)  :  (set([1, 2]), '->'), 
            (frozenset([2]), frozenset([1]), 1)    :  (set([1]), '->')}

    assert bsuccessors3((frozenset([2, 4]), frozenset([3, 5]), 1)) == {
            (frozenset([2, 4, 5]), frozenset([3]), 0)   :  (set([5]), '<-'), 
            (frozenset([2, 3, 4, 5]), frozenset([]), 0) :  (set([3, 5]), '<-'), 
            (frozenset([2, 3, 4]), frozenset([5]), 0)   :  (set([3]), '<-')}
    return 'tests pass'

print test()

tests pass


## Peter's

![ps1](img/ps1.png)

# More Pour Problem

below is simple pour question's successor code.

![pour](img/pour.png)

In [83]:
# In this problem, you will solve the pouring problem for an arbitrary
# number of glasses. Write a function, more_pour_problem, that takes
# as input capacities, goal, and (optionally) start. This function should
# return a path of states and actions.
#
# Capacities is a tuple of numbers, where each number represents the
# volume of a glass.
#
# Goal is the desired volume and start is a tuple of the starting levels
# in each glass. Start defaults to None (all glasses empty).
#
# The returned path should look like [state, action, state, action, ... ]
# where state is a tuple of volumes and action is one of ('fill', i),
# ('empty', i), ('pour', i, j) where i and j are indices indicating the
# glass number.

import copy
import itertools


def more_pour_problem(capacities, goal, start=None):
  """The first argument is a tuple of capacities (numbers) of glasses; the
  goal is a number which we must achieve in some glass.  start is a tuple
  of starting levels for each glass; if None, that means 0 for all.
  Start at start state and follow successors until we reach the goal.
  Keep track of frontier and previously explored; fail when no frontier.
  On success return a path: a [state, action, state2, ...] list, where an
  action is one of ('fill', i), ('empty', i), ('pour', i, j), where
  i and j are indices indicating the glass number."""

  # your code here
  def is_done(state):
    return goal in state

  def p_successors(state):
    d = {}
    pairs = [i for i in itertools.combinations(range(len(capacities)), 2)]
    for (i, j) in pairs:
      d2 = successor_2_cup(i, j, state)
      d.update(d2)
    #print d
    return d

  def replace(seq, i, val):
    # modify one item, and return a copy of seq in same type.
    s = list(seq)
    s[i] = val
    return type(seq)(s)

  def successor_2_cup(i, j, state):
    # (x, y) is glass levels; X and Y are glass capacities
    x = state[i]
    y = state[j]
    X = capacities[i]
    Y = capacities[j]
    assert x <= X and y <= Y

    d = {}
    if y + x <= Y:
      s = replace(state, i, 0)
      s = replace(s, j, y + x)
      d[s] = 'pour', i, j
    else:
      s = replace(state, i, x - (Y - y))
      s = replace(s, j, Y)
      d[s] = 'pour', i, j

    if x+y <= X:
      s = replace(state, i, x+y)
      s = replace(s, j, 0)
      d[s] = 'pour', j, i
    else:
      s = replace(state, i, X)
      s = replace(s, j, y-(X-x))
      d[s] = 'pour', j, i

    d[replace(state, i, X)] = 'fill', i
    d[replace(state, j, Y)] = 'fill', j
    d[replace(state, i, 0)] = 'empty', i
    d[replace(state, j, 0)] = 'empty', j
    
    return d

  if start is None:
    start = (0, ) * len(capacities)
  return shortest_path_search(start, p_successors, is_done)


def shortest_path_search(start, successors, is_goal):
  """Find the shortest path from start state to a state
  such that is_goal(state) is true."""
  if is_goal(start):
    return [start]
  explored = set()
  frontier = [[start]]
  while frontier:
    path = frontier.pop(0)
    s = path[-1]
    for (state, action) in successors(s).items():
      if state not in explored:
        explored.add(state)
        path2 = path + [action, state]
        if is_goal(state):
          #print path2
          return path2
        else:
          frontier.append(path2)
  return Fail


Fail = []


def test_more_pour():
  assert more_pour_problem((1, 2, 4, 8), 4) == [
    (0, 0, 0, 0), ('fill', 2), (0, 0, 4, 0)]

  assert more_pour_problem((1, 2, 4, 8), 3) == [
    (0, 0, 0, 0), ('fill', 2), (0, 0, 4, 0), ('pour', 2, 0), (1, 0, 3, 0)]

  assert more_pour_problem((1, 2, 4), 3) == [
      (0, 0, 0), ('fill', 2), (0, 0, 4), ('pour', 2, 0), (1, 0, 3)]

  starbucks = (8, 12, 16, 20, 24)
  assert not any(more_pour_problem(starbucks, odd) for odd in (3, 5, 7, 9))
  assert all(more_pour_problem((1, 3, 9, 27), n) for n in range(28))
  assert more_pour_problem((1, 3, 9, 27), 28) == []
  return 'test_more_pour passes'


print test_more_pour()

test_more_pour passes


In [34]:
import itertools
[i for i in itertools.combinations(range(4), 2)]

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]

In [47]:
l = (0,)
print l * 5

(0, 0, 0, 0, 0)


In [76]:
def replace(seq, i, val):
    # modify one item, and return a copy of seq in same type.
    s = list(seq)
    s[i] = val
    return type(seq)(s)

s1 = (1,2,3,4)
s2 = replace(s1, 1, 1)
print s1
print s2


s3 = (1,2,3,4)
#s4 = s3 # TypeError: 'tuple' object does not support item assignment

(1, 2, 3, 4)
(1, 1, 3, 4)


In [81]:
import copy
s3 = (1,2,3,4)

s4 = copy.deepcopy(s3)

s4 = list(s4)
s4[1] = 5

print 's3', s3
print 's4 type is list:', s4

s4 = type(s3)(s4)
print 'converted to set:', s4

print type(s3)

s3 (1, 2, 3, 4)
s4 type is list: [1, 5, 3, 4]
converted to set: (1, 5, 3, 4)
<type 'tuple'>


# Tricks I learnt

- `replace` to deep copy a sequence to list, modify it, cast back to original type
- `(0,) * 3` is to generate `(0,0,0)`
- goal for pour is just any cup contains the result.

### Peter's solution

![more](img/morep.png)

# Subway Planning

In [2]:
s = 'dadsa dasdas sdada'
print s.split()

['dadsa', 'dasdas', 'sdada']


In [4]:
s = {2:3}
d = {3:5}
s.update(d)
print s

{2: 3, 3: 5}


In [12]:
dd = [1,2,3]
print dd[-2]
print dd[:-1]

2
[1, 2]


In [14]:
# -----------------
# User Instructions
#
# Write a function, subway, that takes lines as input (read more about
# the **lines notation in the instructor comments box below) and returns
# a dictionary of the form {station:{neighbor:line, ...}, ... }
#
# For example, when calling subway(boston), one of the entries in the
# resulting dictionary should be 'foresthills': {'backbay': 'orange'}.
# This means that foresthills only has one neighbor ('backbay') and
# that neighbor is on the orange line. Other stations have more neighbors:
# 'state', for example, has 4 neighbors.
#
# Once you've defined your subway function, you can define a ride and
# longest_ride function. ride(here, there, system) takes as input
# a starting station (here), a destination station (there), and a subway
# system and returns the shortest path.
#
# longest_ride(system) returns the longest possible ride in a given
# subway system.

# -------------
# Grading Notes
#
# The subway() function will not be tested directly, only ride() and
# longest_ride() will be explicitly tested. If your code passes the
# assert statements in test_ride(), it should be marked correct.

import pprint as pp
import itertools


def find_neighbours(cities, color, search):
  """Find all neighbours for a city.

  :param cities: list of cities in same line color.
  :param color: the line color
  :param search: the city to search
  :return: {'backbay': 'orange', 'sdada':'dadsa'}
  """
  index = cities.index(search)
  d = {}
  if index == 0:
    d[cities[1]] = color
  elif index == len(cities) - 1:
    # If last item, get 2nd last.
    d[cities[-2]] = color
  elif index == -1:
    # err
    return {}
  else:
    d[cities[index-1]] = color
    d[cities[index+1]] = color

  #pp.pprint(d)
  return d

def subway(**lines):
  """Define a subway map. Input is subway(linename='station1 station2...'...).
  Convert that and return a dict of the form: {station:{neighbor:line,...},...}"""
  # your code here
  d = {}  # values are list of strings. Similar to lines.
  unique_cities = set()
  results = {}
  for k, v in lines.items():
    cities = v.split()
    d[k] = cities
    unique_cities.update(cities)

  for c in unique_cities:
    city_dict = {}
    for line in d.keys():
      if c in d[line]:
        neighbours = find_neighbours(d[line], line, c)
        city_dict.update(neighbours)

    results[c] = city_dict

  # pp.pprint(results)
  return results


boston = subway(
  blue='bowdoin government state aquarium maverick airport suffolk revere wonderland',
  orange='oakgrove sullivan haymarket state downtown chinatown tufts backbay foresthills',
  green='lechmere science north haymarket government park copley kenmore newton riverside',
  red='alewife davis porter harvard central mit charles park downtown south umass mattapan')


def ride(here, there, system=boston):
  # Return a path on the subway system from here to there.
  # your code here
  def reach_dst(state):
    return state == there

  def path_successors(state):
    return boston[state] if state in boston else {}

  return shortest_path_search(here, successors=path_successors, is_goal=reach_dst)


Fail = []

def longest_ride(system):
  """Return the longest possible 'shortest path'
  ride between any two stops in the system."""
  # your code here
  all_paths = []
  unique_cities = boston.keys()
  for here, there in itertools.combinations(unique_cities, 2):
    path = ride(here, there)
    #print path
    all_paths.append(path)

  return max(all_paths, key=len)

def shortest_path_search(start, successors, is_goal):
  """Find the shortest path from start state to a state
  such that is_goal(state) is true."""
  if is_goal(start):
    return [start]
  explored = set()  # set of states we have visited
  frontier = [[start]]  # ordered list of paths we have blazed
  while frontier:
    path = frontier.pop(0)
    s = path[-1]
    for (state, action) in successors(s).items():
      if state not in explored:
        explored.add(state)
        path2 = path + [action, state]
        if is_goal(state):
          return path2
        else:
          frontier.append(path2)
  return []


def path_states(path):
  # Return a list of states in this path.
  return path[0::2]


def path_actions(path):
  # Return a list of actions in this path.
  return path[1::2]


def test_subway_func():
  boston

#test_subway_func()

def test_ride():
  assert ride('mit', 'government') == [
    'mit', 'red', 'charles', 'red', 'park', 'green', 'government']
  assert ride('mattapan', 'foresthills') == [
    'mattapan', 'red', 'umass', 'red', 'south', 'red', 'downtown',
    'orange', 'chinatown', 'orange', 'tufts', 'orange', 'backbay', 'orange', 'foresthills']
  assert ride('newton', 'alewife') == [
    'newton', 'green', 'kenmore', 'green', 'copley', 'green', 'park', 'red', 'charles', 'red',
    'mit', 'red', 'central', 'red', 'harvard', 'red', 'porter', 'red', 'davis', 'red', 'alewife']

  assert (path_states(longest_ride(boston)) == [
    'wonderland', 'revere', 'suffolk', 'airport', 'maverick', 'aquarium', 'state', 'downtown', 'park',
    'charles', 'mit', 'central', 'harvard', 'porter', 'davis', 'alewife'] or
          path_states(longest_ride(boston)) == [
            'alewife', 'davis', 'porter', 'harvard', 'central', 'mit', 'charles',
            'park', 'downtown', 'state', 'aquarium', 'maverick', 'airport', 'suffolk', 'revere', 'wonderland'])
  assert len(path_states(longest_ride(boston))) == 16
  return 'test_ride passes'


print test_ride()


"""
{'airport': {'maverick': 'blue', 'suffolk': 'blue'},
 'alewife': {'davis': 'red'},
 'aquarium': {'maverick': 'blue', 'state': 'blue'},
 'backbay': {'foresthills': 'orange', 'tufts': 'orange'},
 'bowdoin': {'government': 'blue'},
 'central': {'harvard': 'red', 'mit': 'red'},
 'charles': {'mit': 'red', 'park': 'red'},
 'chinatown': {'downtown': 'orange', 'tufts': 'orange'},
 'copley': {'kenmore': 'green', 'park': 'green'},
 'davis': {'alewife': 'red', 'porter': 'red'},
 'downtown': {'chinatown': 'orange',
              'park': 'red',
              'south': 'red',
              'state': 'orange'},
 'foresthills': {'backbay': 'orange'},
 'government': {'bowdoin': 'blue',
                'haymarket': 'green',
                'park': 'green',
                'state': 'blue'},
 'harvard': {'central': 'red', 'porter': 'red'},
 'haymarket': {'government': 'green',
               'north': 'green',
               'state': 'orange',
               'sullivan': 'orange'},
 'kenmore': {'copley': 'green', 'newton': 'green'},
 'lechmere': {'science': 'green'},
 'mattapan': {'umass': 'red'},
 'maverick': {'airport': 'blue', 'aquarium': 'blue'},
 'mit': {'central': 'red', 'charles': 'red'},
 'newton': {'kenmore': 'green', 'riverside': 'green'},
 'north': {'haymarket': 'green', 'science': 'green'},
 'oakgrove': {'sullivan': 'orange'},
 'park': {'charles': 'red',
          'copley': 'green',
          'downtown': 'red',
          'government': 'green'},
 'porter': {'davis': 'red', 'harvard': 'red'},
 'revere': {'suffolk': 'blue', 'wonderland': 'blue'},
 'riverside': {'newton': 'green'},
 'science': {'lechmere': 'green', 'north': 'green'},
 'south': {'downtown': 'red', 'umass': 'red'},
 'state': {'aquarium': 'blue',
           'downtown': 'orange',
           'government': 'blue',
           'haymarket': 'orange'},
 'suffolk': {'airport': 'blue', 'revere': 'blue'},
 'sullivan': {'haymarket': 'orange', 'oakgrove': 'orange'},
 'tufts': {'backbay': 'orange', 'chinatown': 'orange'},
 'umass': {'mattapan': 'red', 'south': 'red'},
 'wonderland': {'revere': 'blue'}}
"""

test_ride passes


"\n{'airport': {'maverick': 'blue', 'suffolk': 'blue'},\n 'alewife': {'davis': 'red'},\n 'aquarium': {'maverick': 'blue', 'state': 'blue'},\n 'backbay': {'foresthills': 'orange', 'tufts': 'orange'},\n 'bowdoin': {'government': 'blue'},\n 'central': {'harvard': 'red', 'mit': 'red'},\n 'charles': {'mit': 'red', 'park': 'red'},\n 'chinatown': {'downtown': 'orange', 'tufts': 'orange'},\n 'copley': {'kenmore': 'green', 'park': 'green'},\n 'davis': {'alewife': 'red', 'porter': 'red'},\n 'downtown': {'chinatown': 'orange',\n              'park': 'red',\n              'south': 'red',\n              'state': 'orange'},\n 'foresthills': {'backbay': 'orange'},\n 'government': {'bowdoin': 'blue',\n                'haymarket': 'green',\n                'park': 'green',\n                'state': 'blue'},\n 'harvard': {'central': 'red', 'porter': 'red'},\n 'haymarket': {'government': 'green',\n               'north': 'green',\n               'state': 'orange',\n               'sullivan': 'orange'},\n

## Peter's solution

![solution](img/subway.png)

![subway](img/sub.png)